## Загрузим нужные библиотеки

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!cp /content/drive/MyDrive/train_dataset_train.csv /content/train.csv
!cp /content/drive/MyDrive/test_dataset_test.csv /content/test.csv

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Якутск.zip

Archive:  /content/drive/MyDrive/Якутск.zip
  inflating: Повышение_baseline.ipynb  
  inflating: Республика Саха (Якутия).pdf  
  inflating: Calls.csv               
  inflating: ConnectionTime.csv      
  inflating: Education.csv           
  inflating: SKUD.csv                
  inflating: Tasks.csv               
  inflating: TimenNetwork.csv        
  inflating: WorkingDay.csv          


In [ ]:
import pandas as pd
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import catboost as cb
%matplotlib inline

In [ ]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
df_skud = pd.read_csv('/content/SKUD.csv')
df_calls = pd.read_csv("/content/Calls.csv")
df_tasks = pd.read_csv('/content/Tasks.csv')
df_connection_time = pd.read_csv("/content/ConnectionTime.csv")
df_working_day = pd.read_csv('/content/WorkingDay.csv')
df_ed = pd.read_csv("/content/Education.csv")
df_network = pd.read_csv('/content/TimenNetwork.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#@title EDA & Preparing Data
for i in ['df_skud', 'df_calls', 'df_tasks', 'df_connection_time', 'df_working_day', 'df_network']:
  print(i, end=':\t')
  eval(f'print({i}["id"].nunique(), end="\t")')

df_skud:	865	df_calls:	1071	df_tasks:	1272	df_connection_time:	1588	df_working_day:	1495	df_network:	1491	

In [ ]:
SEED = 0xCAFEC0DE

random.seed(SEED)
np.random.seed(SEED)

In [ ]:
change_division = lambda x: 1 if x == 1 or x == 2 else 0
change_job = lambda x: 1 if x == 0 or x == 2 else 0 

In [ ]:
df_train['division'] = df_train['type'].apply(change_division)
df_train['job'] = df_train['type'].apply(change_job)

In [ ]:
df_train['division'].value_counts()

1    372
0    178
Name: division, dtype: int64

In [ ]:
df_train['job'].value_counts()

1    451
0     99
Name: job, dtype: int64

In [ ]:
len(df_test)

262

In [ ]:
df_test.tail()

,id,type
257,ОРГ1-02893,0
258,ОРГ2-01077,0
259,ОРГ1-00469,0
260,ОРГ1-02060,0
261,ОРГ2-05042,0


## Рассмотрим датасет по ближе

In [ ]:
outlier_detection = lambda df, col: df.loc[(df[col].median() - df[col].mad() > df[col]) | (df[col] > df[col].median() + df[col].mad())]

In [ ]:
fdrop = lambda df, col: outlier_detection(df, col).index

In [ ]:
df_network.head()

,Вых/Будни,monitor_Time,startTime,id
0,Будни,300,"2021-08-16 00:00:00,000",ОРГ1-01402
1,Будни,300,"2021-08-18 00:00:00,000",ОРГ1-01402
2,Будни,300,"2021-08-19 00:00:00,000",ОРГ1-01402
3,Будни,300,"2021-08-23 00:00:00,000",ОРГ1-01402
4,Будни,300,"2021-08-26 00:00:00,000",ОРГ1-01402


In [ ]:
df_network.groupby('Вых/Будни')['monitor_Time'].mean()

Вых/Будни
Будни           16823.924388
Выходные дни     7937.493724
Name: monitor_Time, dtype: float64

In [ ]:
len(df_network)

216110

In [ ]:
med = df_network.groupby('Вых/Будни')['monitor_Time'].median()

In [ ]:
mad = df_network.groupby('Вых/Будни')['monitor_Time'].mad()

In [ ]:
med#, mad

Вых/Будни
Будни           18004.0
Выходные дни     5400.0
Name: monitor_Time, dtype: float64

In [ ]:
outliers = []
grpb = df_network.groupby('Вых/Будни')['monitor_Time']
for i in ['Будни', "Выходные дни"]:
  outliers.append(df_network.loc[( grpb.median()[i] - grpb.mad()[i] > df_network['monitor_Time']) | (df_network['monitor_Time'] > grpb.median()[i] + grpb.mad()[i])])

In [ ]:
check4index = lambda i, idx: 0 if i in idx else 1

In [ ]:
outliers

[           Вых/Будни  monitor_Time                startTime          id
 0              Будни           300  2021-08-16 00:00:00,000  ОРГ1-01402
 1              Будни           300  2021-08-18 00:00:00,000  ОРГ1-01402
 2              Будни           300  2021-08-19 00:00:00,000  ОРГ1-01402
 3              Будни           300  2021-08-23 00:00:00,000  ОРГ1-01402
 4              Будни           300  2021-08-26 00:00:00,000  ОРГ1-01402
 ...              ...           ...                      ...         ...
 216105  Выходные дни           300  2021-11-05 00:00:00,000  ОРГ1-01770
 216106  Выходные дни           300  2021-12-26 00:00:00,000  ОРГ1-02799
 216107  Выходные дни           300  2021-11-06 00:00:00,000  ОРГ1-00933
 216108  Выходные дни           300  2021-11-28 00:00:00,000  ОРГ1-01630
 216109  Выходные дни           300  2021-10-24 00:00:00,000  ОРГ1-03000
 
 [108917 rows x 4 columns],
            Вых/Будни  monitor_Time                startTime          id
 19681          Будни

In [ ]:
df_network['is_outlier_Будни'] = [check4index(i, outliers[0]) for i in range(len(df_network))]
df_network['is_outlier_Выходные'] = [check4index(i, outliers[1]) for i in range(len(df_network))]

In [ ]:
df_network['monitor_Time'] = round(df_network['monitor_Time']/60, 3)

In [ ]:
df_N = pd.DataFrame()

In [ ]:
# df_N['Будни_outliers'] = df_network.groupby('id')['is_outlier_Будни'].sum()
df_N['Выходные_outliers'] = df_network.groupby('id')['is_outlier_Выходные'].sum()

In [ ]:
df_network = df_network.drop(fdrop(df_network, 'monitor_Time'))

In [ ]:
stats = ['sum', 'median', 'count', 'std']
df_N[[f'monitor_Time_{i}' for i in stats]] = df_network.groupby('id')['monitor_Time'].agg(stats)

In [ ]:
tmp = df_network.groupby(['id'])['monitor_Time'].agg(['min', 'max'])
df_N['frequency_monitor_Time'] = df_N['monitor_Time_count'] / (tmp['max'] - tmp['min'] + 1)

In [ ]:
df_N.head()

,Выходные_outliers,monitor_Time_sum,monitor_Time_median,monitor_Time_count,monitor_Time_std,frequency_monitor_Time
id,,,,,,
ОРГ1-00004,354,43687.554,275.0750,164.0,86.635999,0.527358
ОРГ1-00028,278,69022.153,345.2500,208.0,76.131632,0.679146
ОРГ1-00030,277,41290.286,355.0500,127.0,89.421958,0.414649
ОРГ1-00044,157,15349.118,385.2665,44.0,91.012486,0.148548
ОРГ1-00046,267,32028.834,255.0170,123.0,89.868871,0.395244


In [ ]:
df_working_day.head()

,startTime,activeTime,Вых/Будни,monitorTime,id
0,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01553
1,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112
2,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112
3,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01846
4,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01846


In [ ]:
df_WD = pd.DataFrame()

In [ ]:
idx = outlier_detection(df_working_day, 'activeTime').index
df_working_day['is_outlier_activeTime'] = [check4index(i, idx) for i in range(len(df_working_day))]
idx = outlier_detection(df_working_day, 'monitorTime').index
df_working_day['is_outlier_monitorTime'] = [check4index(i, idx) for i in range(len(df_working_day))]

In [ ]:
df_WD['activeTime_outliers'] = df_working_day.groupby('id')['is_outlier_activeTime'].sum()
df_WD['monitorTime_outliers'] = df_working_day.groupby('id')['is_outlier_monitorTime'].sum()

In [ ]:
df_working_day = df_working_day.drop(fdrop(df_working_day, 'activeTime') | fdrop(df_working_day, 'monitorTime'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  """Entry point for launching an IPython kernel.


In [ ]:
stats = ['sum', 'median', 'count', 'std']

In [ ]:
df_WD[[f'activeTime_{i}' for i in stats]] = df_working_day.groupby('id')['activeTime'].agg(stats)
df_WD[[f'monitorTime_{i}' for i in stats]] = df_working_day.groupby('id')['monitorTime'].agg(stats)
tmp = {'activeTime':df_working_day.groupby('id')['activeTime'].count().agg(['min', 'max']), 'monitorTime':df_working_day.groupby('id')['monitorTime'].count().agg(['min', 'max'])}
df_WD['frequency_activeTime'] = df_WD['activeTime_count'] / (tmp['activeTime']['max'] - tmp['activeTime']['min'] + 1)
# df_WD['activeTime_quantile_0.25'] = df_working_day.groupby('id')['activeTime'].quantile(0.25)
# df_WD['activeTime_quantile_0.75'] = df_working_day.groupby('id')['activeTime'].quantile(0.75)
# df_WD['monitorTime_quantile_0.25'] = df_working_day.groupby('id')['monitorTime'].quantile(0.25)
# df_WD['monitorTime_quantile_0.75'] = df_working_day.groupby('id')['monitorTime'].quantile(0.75)
df_WD['frequency_monitorTime'] = df_WD['monitorTime_count'] / (tmp['monitorTime']['max'] - tmp['monitorTime']['min'] + 1)
# df_WD['activeTime_S'] = df_working_day.groupby('id')['activeTime'].apply(f_sum)
# df_WD['monitorTime_S'] = df_working_day.groupby('id')['monitorTime'].apply(f_sum)
# df_WD['activeTime_S'] = df_working_day.groupby('id')['activeTime'].apply(pd.Series.mode)
# df_WD['monitorTime_S'] = df_working_day.groupby('id')['monitorTime'].apply(pd.Series.mode)

In [ ]:
df_tasks['Вид документа'].value_counts()

Служебная записка                              219740
Входящий                                        85972
Исходящий                                       76326
Приказ                                          43382
Внутренний документ                             27526
Договор                                         21082
Дополнительное соглашение                       20128
Акт                                             17999
Распоряжение                                     5898
Файл                                             5704
Протокол                                         3519
Исходящий РМЗ                                    2748
Внутренний документ для БП синхронизация 1С      2201
Запрос на внесение изменений                      665
Отчет по ВА                                       253
Name: Вид документа, dtype: int64

In [ ]:
df_tasks.head()

,Статус по просрочке,Срок плановый,"Просрочено, дней",ДлительностьПросрочки,ID задачи,Вид документа,Дата старта задания,Дата завершения задания плановая,Дата завершения задания фактическая,Состояние задания,id
0,Без нарушения срока,NaN,0,без нарушения срока,E1DE844D-EE2D-4C41-AEDF-93F246749F0E,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
1,Без нарушения срока,NaN,0,без нарушения срока,7A92343C-8C9A-46E7-AC81-8F50F95009D0,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
2,Без нарушения срока,NaN,0,без нарушения срока,5CE64E52-D2D1-4DCC-B2C8-34734AA39AC0,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
3,Без нарушения срока,NaN,0,без нарушения срока,7A28F3DD-983F-4127-AB7F-6EDB85A69F1C,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588
4,Без нарушения срока,NaN,0,без нарушения срока,ABFFAF61-12D8-44C2-B1F6-8402D174889E,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588


In [ ]:
df_tasks['Статус по просрочке'].unique()

array(['Без нарушения срока', 'С нарушением срока'], dtype=object)

In [ ]:
df_tasks = df_tasks.drop(fdrop(df_tasks, 'Просрочено, дней'))

In [ ]:
df_tasks['Статус по просрочке'] = df_tasks['Статус по просрочке'].apply(lambda x: 1 if x == 'С нарушением срока' else 0)

In [ ]:
df_tasks.groupby('id')['ID задачи'].nunique()

id
ОРГ1-00004    1115
ОРГ1-00005    2202
ОРГ1-00028     244
ОРГ1-00030      36
ОРГ1-00044     297
              ... 
ОРГ2-06006     394
ОРГ2-06061     108
ОРГ2-06084      27
ОРГ2-06115       5
ОРГ2-06373     134
Name: ID задачи, Length: 1237, dtype: int64

In [ ]:
dummies = pd.get_dummies(df_tasks['Вид документа'])

In [ ]:
dummies

,Акт,Внутренний документ,Внутренний документ для БП синхронизация 1С,Входящий,Договор,Дополнительное соглашение,Запрос на внесение изменений,Исходящий,Исходящий РМЗ,Отчет по ВА,Приказ,Протокол,Распоряжение,Служебная записка,Файл
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536655,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
536656,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
536657,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
536658,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
dummies['id'] = df_tasks['id']

In [ ]:
dummies.groupby('id').sum()

,Акт,Внутренний документ,Внутренний документ для БП синхронизация 1С,Входящий,Договор,Дополнительное соглашение,Запрос на внесение изменений,Исходящий,Исходящий РМЗ,Отчет по ВА,Приказ,Протокол,Распоряжение,Служебная записка,Файл
id,,,,,,,,,,,,,,,
ОРГ1-00004,5.0,182.0,38.0,123.0,132.0,200.0,0.0,5.0,0.0,2.0,146.0,0.0,21.0,250.0,10.0
ОРГ1-00005,1.0,75.0,0.0,72.0,436.0,1001.0,63.0,98.0,0.0,0.0,80.0,0.0,21.0,349.0,5.0
ОРГ1-00028,0.0,182.0,38.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,2.0,4.0,0.0
ОРГ1-00030,9.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,4.0,0.0
ОРГ1-00044,0.0,3.0,0.0,65.0,0.0,0.0,0.0,38.0,0.0,0.0,101.0,0.0,10.0,20.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ОРГ2-06006,0.0,1.0,0.0,57.0,0.0,0.0,0.0,35.0,0.0,0.0,13.0,16.0,0.0,197.0,74.0
ОРГ2-06061,0.0,0.0,0.0,19.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,84.0,0.0
ОРГ2-06084,0.0,0.0,0.0,2.0,4.0,0.0,0.0,6.0,0.0,0.0,2.0,0.0,0.0,13.0,0.0


In [ ]:
df_T = pd.DataFrame()

In [ ]:
df_T['Количество задач'] = df_tasks.groupby('id')['ID задачи'].nunique()

In [ ]:
df_T['Просрочено дней'] = df_tasks.groupby('id')["Просрочено, дней"].sum()

In [ ]:
df_T['Статус по просрочке'] = df_tasks.groupby('id')['Статус по просрочке'].sum()

In [ ]:
dummies.groupby('id').agg('sum').columns == dummies.columns[:-1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
df_T.head(3)

,Количество задач,Просрочено дней,Статус по просрочке
id,,,
ОРГ1-00004,1115,470,88
ОРГ1-00005,2202,269,39
ОРГ1-00028,244,51,9


In [ ]:
df_T['mode_task_type'] = df_tasks.groupby('id')['Вид документа'].agg(pd.Series.mode)

In [ ]:
df_T[[f'type04tasks_{i}' for i in dummies.columns[:-1]]] = dummies.groupby('id').agg('sum')/dummies.groupby('id').agg('count')

In [ ]:
df_T['count'] = dummies.groupby('id')["Акт"].agg('count')

In [ ]:
df_T.head()

,Количество задач,Просрочено дней,Статус по просрочке,mode_task_type,type04tasks_Акт,type04tasks_Внутренний документ,type04tasks_Внутренний документ для БП синхронизация 1С,type04tasks_Входящий,type04tasks_Договор,type04tasks_Дополнительное соглашение,type04tasks_Запрос на внесение изменений,type04tasks_Исходящий,type04tasks_Исходящий РМЗ,type04tasks_Отчет по ВА,type04tasks_Приказ,type04tasks_Протокол,type04tasks_Распоряжение,type04tasks_Служебная записка,type04tasks_Файл,count
id,,,,,,,,,,,,,,,,,,,,
ОРГ1-00004,1115,470,88,Служебная записка,0.004484,0.163229,0.034081,0.110314,0.118386,0.179372,0.00000,0.004484,0.0,0.001794,0.130942,0.0,0.018834,0.224215,0.008969,1115
ОРГ1-00005,2202,269,39,Дополнительное соглашение,0.000454,0.034060,0.000000,0.032698,0.198002,0.454587,0.02861,0.044505,0.0,0.000000,0.036331,0.0,0.009537,0.158492,0.002271,2202
ОРГ1-00028,244,51,9,Внутренний документ,0.000000,0.745902,0.155738,0.028689,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.045082,0.0,0.008197,0.016393,0.000000,244
ОРГ1-00030,36,17,5,Входящий,0.250000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.166667,0.0,0.138889,0.111111,0.000000,36
ОРГ1-00044,297,510,73,Приказ,0.000000,0.010101,0.000000,0.218855,0.000000,0.000000,0.00000,0.127946,0.0,0.000000,0.340067,0.0,0.033670,0.067340,0.040404,297


In [ ]:
df_tasks.groupby('id')['Дата старта задания'].nunique()

id
ОРГ1-00004    241
ОРГ1-00005    251
ОРГ1-00028    123
ОРГ1-00030     28
ОРГ1-00044    149
             ... 
ОРГ2-06006    151
ОРГ2-06061     68
ОРГ2-06084     24
ОРГ2-06115      4
ОРГ2-06373     78
Name: Дата старта задания, Length: 1237, dtype: int64

In [ ]:
df_tasks['Просрочено, дней'].value_counts()

0     454148
1      10209
2       5811
3       4764
4       3728
5       2896
6       2443
7       2133
8       1431
11      1252
12      1174
13      1142
10      1121
9        948
14       939
15       708
17       662
19       625
18       610
20       595
21       580
16       547
22       450
24       417
26       416
25       399
23       367
Name: Просрочено, дней, dtype: int64

In [ ]:
df_tasks['Состояние задания'] = df_tasks['Состояние задания'].apply(lambda x: 1 if x=='Делегировано' else 0)

In [ ]:
df_T['Количество делегированных задач'] = df_tasks.groupby('id')['Состояние задания'].sum()

In [ ]:
df_T.head()

,Количество задач,Просрочено дней,Статус по просрочке,mode_task_type,type04tasks_Акт,type04tasks_Внутренний документ,type04tasks_Внутренний документ для БП синхронизация 1С,type04tasks_Входящий,type04tasks_Договор,type04tasks_Дополнительное соглашение,...,type04tasks_Исходящий,type04tasks_Исходящий РМЗ,type04tasks_Отчет по ВА,type04tasks_Приказ,type04tasks_Протокол,type04tasks_Распоряжение,type04tasks_Служебная записка,type04tasks_Файл,count,Количество делегированных задач
id,,,,,,,,,,,,,,,,,,,,,
ОРГ1-00004,1115,470,88,Служебная записка,0.004484,0.163229,0.034081,0.110314,0.118386,0.179372,...,0.004484,0.0,0.001794,0.130942,0.0,0.018834,0.224215,0.008969,1115,0
ОРГ1-00005,2202,269,39,Дополнительное соглашение,0.000454,0.034060,0.000000,0.032698,0.198002,0.454587,...,0.044505,0.0,0.000000,0.036331,0.0,0.009537,0.158492,0.002271,2202,0
ОРГ1-00028,244,51,9,Внутренний документ,0.000000,0.745902,0.155738,0.028689,0.000000,0.000000,...,0.000000,0.0,0.000000,0.045082,0.0,0.008197,0.016393,0.000000,244,0
ОРГ1-00030,36,17,5,Входящий,0.250000,0.000000,0.000000,0.333333,0.000000,0.000000,...,0.000000,0.0,0.000000,0.166667,0.0,0.138889,0.111111,0.000000,36,0
ОРГ1-00044,297,510,73,Приказ,0.000000,0.010101,0.000000,0.218855,0.000000,0.000000,...,0.127946,0.0,0.000000,0.340067,0.0,0.033670,0.067340,0.040404,297,0


In [ ]:
df_skud.head()

,Дата,Приход.1,Уход.1,Длительность общая,Длительность раб.дня без обеда,Вых/Будни,id
0,"2021-03-01 00:00:00,000","1899-12-30 08:11:00,000","1899-12-30 17:32:00,000","9,35","8,35",Будни,ОРГ1-00791
1,"2021-03-01 00:00:00,000","1899-12-30 08:25:00,000","1899-12-30 17:32:00,000","9,1166666666666671","8,1166666666666671",Будни,ОРГ1-01826
2,"2021-03-01 00:00:00,000","1899-12-30 08:31:00,000","1899-12-30 17:32:00,000","9,0166666666666657","8,0166666666666657",Будни,ОРГ1-00713
3,"2021-03-01 00:00:00,000","1899-12-30 08:31:00,000","1899-12-30 17:32:00,000","9,0166666666666657","8,0166666666666657",Будни,ОРГ1-02127
4,"2021-03-01 00:00:00,000","1899-12-30 08:19:00,000","1899-12-30 17:32:00,000","9,2166666666666668","8,2166666666666668",Будни,ОРГ1-01739


In [ ]:
df_skud['Длительность раб.дня без обеда'] = df_skud['Длительность раб.дня без обеда'].apply(lambda x: float(x.replace(',', '.')))
df_skud['Длительность общая'] = df_skud['Длительность общая'].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
df_S = pd.DataFrame()

In [ ]:
idx = outlier_detection(df_skud, 'Длительность общая').index
df_skud['is_outlier_Длительность общая'] = [check4index(i, idx) for i in range(len(df_skud))]
df_S['Длительность_outliers'] = df_skud.groupby('id')['is_outlier_Длительность общая'].sum()

In [ ]:
df_skud = df_skud.drop(fdrop(df_skud, 'Длительность общая') | fdrop(df_skud, 'Длительность раб.дня без обеда'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  """Entry point for launching an IPython kernel.


In [ ]:
df_skud['Обед'] = df_skud['Длительность общая'] - df_skud['Длительность раб.дня без обеда']

In [ ]:
df_S['Длительность_std'] = df_skud.groupby('id')["Длительность общая"].std()
df_S['Длительность_med'] = df_skud.groupby('id')["Длительность общая"].median()

In [ ]:
df_calls.head(3)

,Date,CallTime,NumberOfCalls,Вид учета времени,InOut,id
0,"2021-08-16 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
1,"2021-09-21 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
2,"2021-01-11 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945


In [ ]:
df_calls['Вид учета времени'].unique()

array(['Будни', nan, 'Выходные дни'], dtype=object)

In [ ]:
df_calls['Вид учета времени'] = df_calls['Вид учета времени'].apply(lambda x: 1 if x == "Выходные дни" else 0)

In [ ]:
df_calls['InOut'].unique()

array(['ToUser', 'FromUser'], dtype=object)

In [ ]:
df_calls['CallTime'] = df_calls['CallTime'].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
len(df_calls)

407867

In [ ]:
df_c = pd.DataFrame()

In [ ]:
idx = (outlier_detection(df_calls, 'NumberOfCalls').index & outlier_detection(df_calls, 'CallTime').index)
df_calls['is_outlier_Calls'] = [check4index(i, idx) for i in range(len(df_calls))]
df_c['is_outlier_Calls'] = df_calls.groupby('id')['is_outlier_Calls'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  """Entry point for launching an IPython kernel.


In [ ]:
df_calls = df_calls.drop(fdrop(df_calls, 'NumberOfCalls') | fdrop(df_calls, 'CallTime'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  """Entry point for launching an IPython kernel.


In [ ]:
df_c['c_T'] = df_calls.groupby('id')['Вид учета времени'].sum()

In [ ]:
df_calls['InOut'] = df_calls['InOut'].apply(lambda x: 1 if x == 'ToUser' else 0)
df_c[['s_C', 'c_C']] = df_calls.groupby('id')['InOut'].agg(['sum', 'count'])

In [ ]:
df_c[[f'CallTime_{i}' for i in stats]] = df_calls.groupby('id')['CallTime'].agg(stats)

In [ ]:
df_calls

,Date,CallTime,NumberOfCalls,Вид учета времени,InOut,id,is_outlier_Calls
0,"2021-08-16 00:00:00,000",0.000278,1,0,1,ОРГ1-01945,1
1,"2021-09-21 00:00:00,000",0.000278,1,0,1,ОРГ1-01945,1
2,"2021-01-11 00:00:00,000",0.000278,1,0,1,ОРГ1-01945,1
3,"2021-01-18 00:00:00,000",0.000278,1,0,1,ОРГ1-01945,1
4,"2021-01-27 00:00:00,000",0.000278,1,0,1,ОРГ1-01945,1
...,...,...,...,...,...,...,...
407862,"2021-11-02 00:00:00,000",0.001389,2,0,1,ОРГ1-02152,1
407863,"2021-11-30 00:00:00,000",0.001389,2,0,1,ОРГ1-01681,1
407864,"2021-12-02 00:00:00,000",0.001389,2,0,1,ОРГ1-01945,1
407865,"2021-12-15 00:00:00,000",0.001389,2,0,1,ОРГ1-00933,1


In [ ]:
df_c.head()

,is_outlier_Calls,c_T,s_C,c_C,CallTime_sum,CallTime_median,CallTime_count,CallTime_std
id,,,,,,,,
ОРГ1-00004,356,6.0,104.0,196.0,18.116111,0.073194,196.0,0.074445
ОРГ1-00005,808,9.0,408.0,582.0,40.304167,0.035139,582.0,0.079231
ОРГ1-00028,432,4.0,202.0,361.0,27.455833,0.053056,361.0,0.067873
ОРГ1-00030,491,1.0,188.0,359.0,21.478889,0.044444,359.0,0.050988
ОРГ1-00044,524,10.0,203.0,396.0,35.456389,0.065278,396.0,0.080137


In [ ]:
df_calls['CallTime'].mean()

0.06842587218032081

In [ ]:
df_calls.median()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


CallTime             0.0425
NumberOfCalls        2.0000
Вид учета времени    0.0000
InOut                1.0000
is_outlier_Calls     1.0000
dtype: float64

In [ ]:

df_calls.mad()

CallTime             0.056281
NumberOfCalls        1.490444
Вид учета времени    0.046133
InOut                0.483163
is_outlier_Calls     0.000000
dtype: float64

In [ ]:
df_connection_time.head(3)

,dateNum,maxLogOff,Нормативное время начала раб.дня,Фактич. время начала раб.дня,Время опоздания,Признак опоздания,Вых/Будни,id
0,"2021-12-15 00:00:00,000","2021-12-15 17:30:27,246","1899-12-30 08:30:00,000","1899-12-30 08:24:18,606",NaN,NaN,Будни,ОРГ1-02782
1,"2021-12-15 00:00:00,000","2021-12-15 16:30:13,330","1899-12-30 08:30:00,000","1899-12-30 01:05:20,513",NaN,NaN,Будни,ОРГ1-01407
2,"2021-12-15 00:00:00,000","2021-12-15 15:27:37,246","1899-12-30 08:30:00,000","1899-12-30 05:47:33,156",NaN,NaN,Будни,ОРГ1-01909


In [ ]:
df_ed.head(3)

,Табельный номер руководителя,id,Вид образования,Специальность
0,NaN,ОРГ1-00131,Высшее образование - бакалавриат,Юриспруденция
1,NaN,ОРГ1-03220,Среднее профессиональное образование,Финансы
2,NaN,ОРГ1-03008,Высшее образование,Юриспруденция


In [ ]:
df_train.head(3)

,id,type,division,job
0,ОРГ1-02050,2,1,1
1,ОРГ1-02783,2,1,1
2,ОРГ2-06173,0,0,1


In [ ]:
df_test.head(3)

,id,type
0,ОРГ1-02649,0
1,ОРГ2-05929,0
2,ОРГ2-05859,0


Узнаем размерность наборов

In [ ]:
print("df_train", df_train.shape)
print("df_calls", df_calls.shape)
print("df_connection_time", df_connection_time.shape)
print("df_ed", df_ed.shape)
print("df_test", df_test.shape)

df_train (550, 4)
df_calls (275647, 7)
df_connection_time (230662, 8)
df_ed (5609, 4)
df_test (262, 2)


### Видоизмение данных

Обьединим два набора с данными

In [ ]:
def change_ed(text):
  if text == 'Высшее образование - бакалавриат' or text == 'Высшее образование - специалитет, магистратура':
    return 'Высшее образование'
  elif text == 'Начальное профессиональное образование' or text == 'Профессиональное обучение' or text == 'Неполное высшее образование' or text == 'Дополнительное профессиональное образование':
    return 'Среднее профессиональное образование'
  elif text == 'Переподготовка':
    return 'Повышение квалификации'
  elif text == 'Среднее (полное) общее образование' or text == 'Основное общее образование' or text == 'Начальное общее образование':
    return 'Среднее общее образование'
  elif text == 'Послевузовское образование':
    return 'Аспирантура'
  else: return text

In [ ]:
df_ed['Вид образования'].unique()

array(['Высшее образование - бакалавриат',
       'Среднее профессиональное образование', 'Высшее образование',
       'Повышение квалификации', nan, 'Переподготовка',
       'Высшее образование - специалитет, магистратура',
       'Начальное профессиональное образование', 'Аспирантура',
       'Среднее общее образование', 'Среднее (полное) общее образование',
       'Начальное общее образование', 'Неполное высшее образование',
       'Основное общее образование', 'Профессиональное обучение',
       'Послевузовское образование',
       'Дополнительное профессиональное образование'], dtype=object)

In [ ]:
# edC = {}
# unique = ['Высшее образование - бакалавриат',
#        'Среднее профессиональное образование', 'Высшее образование',
#        'Повышение квалификации', np.nan, 'Переподготовка',
#        'Высшее образование - специалитет, магистратура',
#        'Начальное профессиональное образование', 'Аспирантура',
#        'Среднее общее образование', 'Среднее (полное) общее образование',
#        'Начальное общее образование', 'Неполное высшее образование',
#        'Основное общее образование', 'Профессиональное обучение',
#        'Послевузовское образование',
#        'Дополнительное профессиональное образование']
# for i in range(df_ed['Вид образования'].nunique() + 1):
#   edC[unique[i]] = change_ed(unique[i])

In [ ]:
df_ed['Вид образования'] = df_ed['Вид образования'].apply(change_ed)

In [ ]:
df_ed['Вид образования'].unique()

array(['Высшее образование', 'Среднее профессиональное образование',
       'Повышение квалификации', nan, 'Аспирантура',
       'Среднее общее образование'], dtype=object)

In [ ]:
df_ed.groupby('id')['Вид образования'].nunique().max()

3

In [ ]:
df_ed.groupby('id')['Вид образования'].apply(lambda x: x.unique()[-1]) # current education

id
ОРГ1-00001                  Повышение квалификации
ОРГ1-00004                  Повышение квалификации
ОРГ1-00005                  Повышение квалификации
ОРГ1-00010                      Высшее образование
ОРГ1-00013                                     NaN
                              ...                 
ОРГ2-09282                      Высшее образование
ОРГ2-09283                      Высшее образование
ОРГ2-09284    Среднее профессиональное образование
ОРГ2-09285                      Высшее образование
ОРГ2-09286                      Высшее образование
Name: Вид образования, Length: 4788, dtype: object

In [ ]:
df_ed.groupby('id')['Специальность'].nunique().max() # Максимальное количество специальностей - следует сделать 6 колонок со специальностями

6

In [ ]:
df_ed.groupby('id')['Табельный номер руководителя'].count()

id
ОРГ1-00001    0
ОРГ1-00004    6
ОРГ1-00005    6
ОРГ1-00010    0
ОРГ1-00013    3
             ..
ОРГ2-09282    0
ОРГ2-09283    1
ОРГ2-09284    1
ОРГ2-09285    0
ОРГ2-09286    0
Name: Табельный номер руководителя, Length: 4788, dtype: int64

In [ ]:
df_E = pd.DataFrame()
df_E['count_ed'] = df_ed.groupby('id')['Табельный номер руководителя'].count()
df_E['ed_type_1'] = df_ed.groupby('id')['Вид образования'].apply(lambda x: x.unique()[-1])
df_E['ed_type_2'] = df_ed.groupby('id')['Вид образования'].apply(lambda x: x.unique()[-2] if x.nunique() > 1 else 0)
df_E['ed_type_3'] = df_ed.groupby('id')['Вид образования'].apply(lambda x: x.unique()[-3] if x.nunique() > 2 else 0)
df_E['Руководитель'] = df_ed.groupby('id')['Табельный номер руководителя'].apply(lambda x: x.unique()[0])

In [ ]:
for i in range(6):
  df_E[f'ed_spec_{i}'] = df_ed.groupby('id')['Специальность'].apply(lambda x: x.unique()[-(i + 1)] if x.nunique() > i else 0)

In [ ]:
df_E

,count_ed,ed_type_1,ed_type_2,ed_type_3,Руководитель,ed_spec_0,ed_spec_1,ed_spec_2,ed_spec_3,ed_spec_4,ed_spec_5
id,,,,,,,,,,,
ОРГ1-00001,0,Повышение квалификации,Высшее образование,0,NaN,Организация подготовки проектной документации,Деятельность по проектированию зданий и сооруж...,Химическая технология переработки нефти и газа,0,0,0
ОРГ1-00004,6,Повышение квалификации,Высшее образование,0,ОРГ1-00004,Интегрированная система менеджмента согласно с...,Практические вопросы применения МСФО,NaN,0,0,0
ОРГ1-00005,6,Повышение квалификации,Высшее образование,0,ОРГ1-00005,Безопасность строительства и осуществление стр...,Безопасность строительства. Организация строит...,Интегрированная система менеджмента согласно с...,"Наружные системы и сети электроснабжения, слаб...",Деятельность по проектированию зданий и сооруж...,Электроэнергетические системы и сети
ОРГ1-00010,0,Высшее образование,0,0,NaN,Экономика и организация машиностроительной про...,0,0,0,0,0
ОРГ1-00013,3,NaN,0,0,ОРГ1-00321,Интегрированная система менеджмента согласно с...,Финансы и кредит,Менеджмент в социальной сфере,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
ОРГ2-09282,0,Высшее образование,0,0,NaN,Автомобили и автомобильное хозяйство,0,0,0,0,0
ОРГ2-09283,1,Высшее образование,0,0,ОРГ2-05294,Бухгалтерский учет и аудит,0,0,0,0,0
ОРГ2-09284,1,Среднее профессиональное образование,0,0,ОРГ2-09100,"Слесарь по ремонту автомобилей, электромонтер ...",0,0,0,0,0


In [ ]:
df_test.shape

(262, 2)

In [ ]:
df_train.head()

,id,type,division,job
0,ОРГ1-02050,2,1,1
1,ОРГ1-02783,2,1,1
2,ОРГ2-06173,0,0,1
3,ОРГ2-05359,2,1,1
4,ОРГ1-00803,2,1,1


In [ ]:
df_test.shape

(262, 2)

Оставим только столбец - опоздания сотрудников

In [ ]:
df_connection_time.head()

,dateNum,maxLogOff,Нормативное время начала раб.дня,Фактич. время начала раб.дня,Время опоздания,Признак опоздания,Вых/Будни,id
0,"2021-12-15 00:00:00,000","2021-12-15 17:30:27,246","1899-12-30 08:30:00,000","1899-12-30 08:24:18,606",NaN,NaN,Будни,ОРГ1-02782
1,"2021-12-15 00:00:00,000","2021-12-15 16:30:13,330","1899-12-30 08:30:00,000","1899-12-30 01:05:20,513",NaN,NaN,Будни,ОРГ1-01407
2,"2021-12-15 00:00:00,000","2021-12-15 15:27:37,246","1899-12-30 08:30:00,000","1899-12-30 05:47:33,156",NaN,NaN,Будни,ОРГ1-01909
3,"2021-12-15 00:00:00,000","2021-12-15 20:35:14,833","1899-12-30 08:30:00,000","1899-12-30 08:28:43,956",NaN,NaN,Будни,ОРГ1-00254
4,"2021-12-15 00:00:00,000","2021-12-15 17:32:16,913","1899-12-30 08:30:00,000","1899-12-30 08:16:02,896",NaN,NaN,Будни,ОРГ1-02543


In [ ]:
sum(df_connection_time.drop(df_connection_time.loc[df_connection_time['Вых/Будни'] == 'Будни'].index)['Признак опоздания'].notna())

8440

In [ ]:
df_connection_time['Время опоздания'].nunique()

100085

In [ ]:
df_connection_time['Время опоздания'] = df_connection_time['Время опоздания'].apply(lambda x: float(str(x).replace(',', '.')))

In [ ]:
df_connection_time_res = df_connection_time.drop(df_connection_time.loc[df_connection_time['Вых/Будни'] == 'Выходные дни'].index)[['id', 'Признак опоздания']]
df_connection_time_res = df_connection_time_res[df_connection_time_res["Признак опоздания"].notna()]
tmp = df_connection_time_res["Признак опоздания"].notna()
df_connection_time_res.head()

,id,Признак опоздания
127561,ОРГ1-01589,Опоздание
127562,ОРГ1-01589,Опоздание
127563,ОРГ1-01589,Опоздание
127564,ОРГ1-01589,Опоздание
127565,ОРГ1-01589,Опоздание


In [ ]:
tmp.index

Int64Index([127561, 127562, 127563, 127564, 127565, 127566, 127567, 127568,
            127569, 127570,
            ...
            230652, 230653, 230654, 230655, 230656, 230657, 230658, 230659,
            230660, 230661],
           dtype='int64', length=69080)

In [ ]:
df_connection_time_res =  df_connection_time.groupby('id')['Время опоздания'].agg(stats)
df_connection_time_res = df_connection_time_res.rename(columns={"sum": "Время опоздания_sum", 
                                                                'median':'Время опоздания_median',
                                                                'count':'Время опоздания_count',
                                                                'std':'Время опоздания_std'})
df_connection_time_res.head(3)

,Время опоздания_sum,Время опоздания_median,Время опоздания_count,Время опоздания_std
id,,,,
ОРГ1-00004,1077.125896,1.410584,314,4.283017
ОРГ1-00028,121.901361,0.311683,54,3.370791
ОРГ1-00030,237.536299,0.116452,132,3.392649


In [ ]:
df_E

,count_ed,ed_type_1,ed_type_2,ed_type_3,Руководитель,ed_spec_0,ed_spec_1,ed_spec_2,ed_spec_3,ed_spec_4,ed_spec_5
id,,,,,,,,,,,
ОРГ1-00001,0,Повышение квалификации,Высшее образование,0,NaN,Организация подготовки проектной документации,Деятельность по проектированию зданий и сооруж...,Химическая технология переработки нефти и газа,0,0,0
ОРГ1-00004,6,Повышение квалификации,Высшее образование,0,ОРГ1-00004,Интегрированная система менеджмента согласно с...,Практические вопросы применения МСФО,NaN,0,0,0
ОРГ1-00005,6,Повышение квалификации,Высшее образование,0,ОРГ1-00005,Безопасность строительства и осуществление стр...,Безопасность строительства. Организация строит...,Интегрированная система менеджмента согласно с...,"Наружные системы и сети электроснабжения, слаб...",Деятельность по проектированию зданий и сооруж...,Электроэнергетические системы и сети
ОРГ1-00010,0,Высшее образование,0,0,NaN,Экономика и организация машиностроительной про...,0,0,0,0,0
ОРГ1-00013,3,NaN,0,0,ОРГ1-00321,Интегрированная система менеджмента согласно с...,Финансы и кредит,Менеджмент в социальной сфере,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
ОРГ2-09282,0,Высшее образование,0,0,NaN,Автомобили и автомобильное хозяйство,0,0,0,0,0
ОРГ2-09283,1,Высшее образование,0,0,ОРГ2-05294,Бухгалтерский учет и аудит,0,0,0,0,0
ОРГ2-09284,1,Среднее профессиональное образование,0,0,ОРГ2-09100,"Слесарь по ремонту автомобилей, электромонтер ...",0,0,0,0,0


In [ ]:
cols = ['Руководитель', 'ed_type_1', 'ed_spec_1']
tmp = df_E[cols]

In [ ]:
df_N = pd.merge(df_N, tmp, on='id', how='left')
df_N.head(3)

,Выходные_outliers,monitor_Time_sum,monitor_Time_median,monitor_Time_count,monitor_Time_std,frequency_monitor_Time,Руководитель,ed_type_1,ed_spec_1
id,,,,,,,,,
ОРГ1-00004,354,43687.554,275.075,164.0,86.635999,0.527358,ОРГ1-00004,Повышение квалификации,Практические вопросы применения МСФО
ОРГ1-00028,278,69022.153,345.250,208.0,76.131632,0.679146,ОРГ1-02529,Повышение квалификации,бухгалтерский учет в коммерческих организациях
ОРГ1-00030,277,41290.286,355.050,127.0,89.421958,0.414649,ОРГ1-02529,Повышение квалификации,0


In [ ]:
for i in range(len(cols)):
  df_N[f"Mean_Monitor_Time_by_ed_{cols[i]}_median"] = ( df_N.groupby(cols[i])["monitor_Time_median"].transform("mean"))
  df_N[f"Mean_Monitor_Time_by_ed_{cols[i]}_outliers"] = ( df_N.groupby(cols[i])["Выходные_outliers"].transform("mean"))
  df_N[f"Mean_Monitor_Time_by_ed_{cols[i]}_frequency"] = ( df_N.groupby(cols[i])["frequency_monitor_Time"].transform("mean"))
  df_N[f"Mean_Monitor_Time_by_ed_{cols[i]}_count"] = ( df_N.groupby(cols[i])["monitor_Time_count"].transform("mean"))

In [ ]:
df_N = df_N.drop(cols, axis=1)

In [ ]:
# df_connection_time_res = df_connection_time_res.drop('Руководитель_x', axis=1)
df_connection_time_res = pd.merge(df_connection_time_res, tmp, on='id', how='left')
df_connection_time_res.head(2)

,Время опоздания_sum,Время опоздания_median,Время опоздания_count,Время опоздания_std,Руководитель,ed_type_1,ed_spec_1
id,,,,,,,
ОРГ1-00004,1077.125896,1.410584,314,4.283017,ОРГ1-00004,Повышение квалификации,Практические вопросы применения МСФО
ОРГ1-00028,121.901361,0.311683,54,3.370791,ОРГ1-02529,Повышение квалификации,бухгалтерский учет в коммерческих организациях


In [ ]:
for i in range(len(cols)):
  df_connection_time_res[f"Mean_Время_опоздания_by_ed_{cols[i]}_sum"] = ( df_connection_time_res.groupby(cols[i])["Время опоздания_sum"].transform("median"))
  df_connection_time_res[f"Mean_Время_опоздания_by_ed_{cols[i]}_median"] = ( df_connection_time_res.groupby(cols[i])["Время опоздания_median"].transform("mean"))
  df_connection_time_res[f"Mean_Время_опоздания_by_ed_{cols[i]}_count"] = ( df_connection_time_res.groupby(cols[i])["Время опоздания_count"].transform("mean"))
  df_connection_time_res[f"Mean_Время_опоздания_by_ed_{cols[i]}_std"] = ( df_connection_time_res.groupby(cols[i])["Время опоздания_std"].transform("mean"))

In [ ]:
df_connection_time_res = df_connection_time_res.drop(cols, axis=1)

In [ ]:
df_connection_time_res.head(3)

,Время опоздания_sum,Время опоздания_median,Время опоздания_count,Время опоздания_std,Mean_Время_опоздания_by_ed_Руководитель_sum,Mean_Время_опоздания_by_ed_Руководитель_median,Mean_Время_опоздания_by_ed_Руководитель_count,Mean_Время_опоздания_by_ed_Руководитель_std,Mean_Время_опоздания_by_ed_ed_type_1_sum,Mean_Время_опоздания_by_ed_ed_type_1_median,Mean_Время_опоздания_by_ed_ed_type_1_count,Mean_Время_опоздания_by_ed_ed_type_1_std,Mean_Время_опоздания_by_ed_ed_spec_1_sum,Mean_Время_опоздания_by_ed_ed_spec_1_median,Mean_Время_опоздания_by_ed_ed_spec_1_count,Mean_Время_опоздания_by_ed_ed_spec_1_std
id,,,,,,,,,,,,,,,,
ОРГ1-00004,1077.125896,1.410584,314,4.283017,1077.125896,1.410584,314.000000,4.283017,78.115002,1.342407,67.786096,2.837479,1077.125896,1.410584,314.000000,4.283017
ОРГ1-00028,121.901361,0.311683,54,3.370791,178.076194,0.665887,152.923077,2.656735,78.115002,1.342407,67.786096,2.837479,121.901361,0.311683,54.000000,3.370791
ОРГ1-00030,237.536299,0.116452,132,3.392649,178.076194,0.665887,152.923077,2.656735,78.115002,1.342407,67.786096,2.837479,76.514424,1.513195,65.342128,2.584162


In [ ]:
df_N.shape

(1491, 18)

In [ ]:
df_train.columns

Index(['id', 'type', 'division', 'job'], dtype='object')

In [ ]:
df_test.shape

(262, 2)

Первая часть фич

In [ ]:
# full_df = pd.DataFrame()
df_full = df_E
df_full = pd.merge(df_full, df_S, on='id', how='left')
df_full = pd.merge(df_full, df_c, on='id', how='left')
df_full = pd.merge(df_full, df_T, on='id', how='left')
df_full = pd.merge(df_full, df_WD, on='id', how='left')
df_full = pd.merge(df_full, df_N, on='id', how='left')
df_full = pd.merge(df_full, df_connection_time_res, on='id', how='left')

Фикс датафреймов для сборки полного датафрейма

In [ ]:
df_S['Руководитель_x'] = df_S.index
df_c['Руководитель_x'] = df_c.index
df_N['Руководитель_x'] = df_N.index
df_E['Руководитель_x'] = df_E.index
df_T['Руководитель_x'] = df_T.index
df_WD['Руководитель_x'] = df_WD.index
df_connection_time_res['Руководитель_x'] = df_connection_time_res.index
df_full['Руководитель_x'] = df_full['Руководитель']
df_full = df_full.drop('Руководитель', axis=1)

Добавление второй части фич

In [ ]:
df_full.columns

Index(['count_ed', 'ed_type_1', 'ed_type_2', 'ed_type_3', 'ed_spec_0',
       'ed_spec_1', 'ed_spec_2', 'ed_spec_3', 'ed_spec_4', 'ed_spec_5',
       'Длительность_outliers', 'Длительность_std', 'Длительность_med',
       'is_outlier_Calls', 'c_T', 's_C', 'c_C', 'CallTime_sum',
       'CallTime_median', 'CallTime_count', 'CallTime_std', 'Количество задач',
       'Просрочено дней', 'Статус по просрочке', 'mode_task_type',
       'type04tasks_Акт', 'type04tasks_Внутренний документ',
       'type04tasks_Внутренний документ для БП синхронизация 1С',
       'type04tasks_Входящий', 'type04tasks_Договор',
       'type04tasks_Дополнительное соглашение',
       'type04tasks_Запрос на внесение изменений', 'type04tasks_Исходящий',
       'type04tasks_Исходящий РМЗ', 'type04tasks_Отчет по ВА',
       'type04tasks_Приказ', 'type04tasks_Протокол',
       'type04tasks_Распоряжение', 'type04tasks_Служебная записка',
       'type04tasks_Файл', 'count', 'Количество делегированных задач',
       'active

In [ ]:
df_T.columns

Index(['Количество задач', 'Просрочено дней', 'Статус по просрочке',
       'mode_task_type', 'type04tasks_Акт', 'type04tasks_Внутренний документ',
       'type04tasks_Внутренний документ для БП синхронизация 1С',
       'type04tasks_Входящий', 'type04tasks_Договор',
       'type04tasks_Дополнительное соглашение',
       'type04tasks_Запрос на внесение изменений', 'type04tasks_Исходящий',
       'type04tasks_Исходящий РМЗ', 'type04tasks_Отчет по ВА',
       'type04tasks_Приказ', 'type04tasks_Протокол',
       'type04tasks_Распоряжение', 'type04tasks_Служебная записка',
       'type04tasks_Файл', 'count', 'Количество делегированных задач',
       'Руководитель_x'],
      dtype='object')

In [ ]:
df_full = pd.merge(df_full, df_S, on='Руководитель_x', how='left')
df_full = pd.merge(df_full, df_c, on='Руководитель_x', how='left')
df_full = pd.merge(df_full, df_WD, on='Руководитель_x', how='left')
df_full = pd.merge(df_full, df_T, on='Руководитель_x', how='left')
df_full = pd.merge(df_full, df_N, on='Руководитель_x', how='left')
df_full = pd.merge(df_full, df_connection_time_res, on='Руководитель_x', how='left')
df_full = pd.merge(df_full, df_E, on='Руководитель_x', how='left')

In [ ]:
# df_full.drop(['ed_spec_1_y', 'ed_spec_2_y', 'ed_spec_3_y', 'ed_spec_4_y', 'ed_spec_5_y', 'ed_type_2_y', 'ed_type_3_y'], axis=1)

Проверка данных на валидность и изменение индекса на исходный

In [ ]:
df_full.head(3)

,count_ed_x,ed_type_1_x,ed_type_2_x,ed_type_3_x,ed_spec_0_x,ed_spec_1_x,ed_spec_2_x,ed_spec_3_x,ed_spec_4_x,ed_spec_5_x,...,ed_type_1_y,ed_type_2_y,ed_type_3_y,Руководитель,ed_spec_0_y,ed_spec_1_y,ed_spec_2_y,ed_spec_3_y,ed_spec_4_y,ed_spec_5_y
0,0,Повышение квалификации,Высшее образование,0,Организация подготовки проектной документации,Деятельность по проектированию зданий и сооруж...,Химическая технология переработки нефти и газа,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,Повышение квалификации,Высшее образование,0,Интегрированная система менеджмента согласно с...,Практические вопросы применения МСФО,NaN,0,0,0,...,Повышение квалификации,Высшее образование,0,ОРГ1-00004,Интегрированная система менеджмента согласно с...,Практические вопросы применения МСФО,NaN,0,0,0
2,6,Повышение квалификации,Высшее образование,0,Безопасность строительства и осуществление стр...,Безопасность строительства. Организация строит...,Интегрированная система менеджмента согласно с...,"Наружные системы и сети электроснабжения, слаб...",Деятельность по проектированию зданий и сооруж...,Электроэнергетические системы и сети,...,Повышение квалификации,Высшее образование,0,ОРГ1-00005,Безопасность строительства и осуществление стр...,Безопасность строительства. Организация строит...,Интегрированная система менеджмента согласно с...,"Наружные системы и сети электроснабжения, слаб...",Деятельность по проектированию зданий и сооруж...,Электроэнергетические системы и сети


In [ ]:
print(*df_full.columns)

count_ed_x ed_type_1_x ed_type_2_x ed_type_3_x ed_spec_0_x ed_spec_1_x ed_spec_2_x ed_spec_3_x ed_spec_4_x ed_spec_5_x Длительность_outliers_x Длительность_std_x Длительность_med_x is_outlier_Calls_x c_T_x s_C_x c_C_x CallTime_sum_x CallTime_median_x CallTime_count_x CallTime_std_x Количество задач_x Просрочено дней_x Статус по просрочке_x mode_task_type_x type04tasks_Акт_x type04tasks_Внутренний документ_x type04tasks_Внутренний документ для БП синхронизация 1С_x type04tasks_Входящий_x type04tasks_Договор_x type04tasks_Дополнительное соглашение_x type04tasks_Запрос на внесение изменений_x type04tasks_Исходящий_x type04tasks_Исходящий РМЗ_x type04tasks_Отчет по ВА_x type04tasks_Приказ_x type04tasks_Протокол_x type04tasks_Распоряжение_x type04tasks_Служебная записка_x type04tasks_Файл_x count_x Количество делегированных задач_x activeTime_outliers_x monitorTime_outliers_x activeTime_sum_x activeTime_median_x activeTime_count_x activeTime_std_x monitorTime_sum_x monitorTime_median_x moni

In [ ]:
len(df_full)

4788

In [ ]:
df_full.index = df_E.index

In [ ]:
df_full.index

Index(['ОРГ1-00001', 'ОРГ1-00004', 'ОРГ1-00005', 'ОРГ1-00010', 'ОРГ1-00013',
       'ОРГ1-00028', 'ОРГ1-00030', 'ОРГ1-00044', 'ОРГ1-00046', 'ОРГ1-00049',
       ...
       'ОРГ2-09277', 'ОРГ2-09278', 'ОРГ2-09279', 'ОРГ2-09280', 'ОРГ2-09281',
       'ОРГ2-09282', 'ОРГ2-09283', 'ОРГ2-09284', 'ОРГ2-09285', 'ОРГ2-09286'],
      dtype='object', name='id', length=4788)

In [ ]:
df_train = pd.merge(df_train, df_full, on='id', how='left')

In [ ]:
# df_full['exp_id'].value_counts()

In [ ]:
df_train.head()

,id,type,division,job,count_ed_x,ed_type_1_x,ed_type_2_x,ed_type_3_x,ed_spec_0_x,ed_spec_1_x,...,ed_type_1_y,ed_type_2_y,ed_type_3_y,Руководитель,ed_spec_0_y,ed_spec_1_y,ed_spec_2_y,ed_spec_3_y,ed_spec_4_y,ed_spec_5_y
0,ОРГ1-02050,2,1,1,0.0,Высшее образование,Среднее профессиональное образование,0,Технологические машины и оборудование,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ОРГ1-02783,2,1,1,1.0,Высшее образование,0,0,Автоматизация технологических процессов и прои...,0,...,Высшее образование,0,0,ОРГ1-01974,Прикладная математика и информатика,0,0,0,0,0
2,ОРГ2-06173,0,0,1,1.0,Высшее образование,0,0,Управление качеством,0,...,Высшее образование,0,0,ОРГ2-01170,Правовое обеспечение государственного и муници...,0,0,0,0,0
3,ОРГ2-05359,2,1,1,1.0,Среднее профессиональное образование,0,0,Технолог-сварщик,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ОРГ1-00803,2,1,1,0.0,Высшее образование,0,0,Журналистика,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_test = pd.merge(df_test, df_full, on='id', how='left')

In [ ]:
df_train.columns

Index(['id', 'type', 'division', 'job', 'count_ed_x', 'ed_type_1_x',
       'ed_type_2_x', 'ed_type_3_x', 'ed_spec_0_x', 'ed_spec_1_x',
       ...
       'ed_type_1_y', 'ed_type_2_y', 'ed_type_3_y', 'Руководитель',
       'ed_spec_0_y', 'ed_spec_1_y', 'ed_spec_2_y', 'ed_spec_3_y',
       'ed_spec_4_y', 'ed_spec_5_y'],
      dtype='object', length=182)

In [ ]:
df_test.shape

(262, 180)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 549
Columns: 182 entries, id to ed_spec_5_y
dtypes: float64(156), int64(3), object(23)
memory usage: 786.3+ KB


Заменим пропуски на нули

In [ ]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
# plt.rcParams['figure.figsize']=(15,15)

# corr = df_train.corr()
# g = sns.heatmap(corr, square = True, annot=True)

## Выделим выборки

In [ ]:
lst_drop = []
for i in df_train.columns:
  if i[0] + i[1] == 'id' or i[0] == 'Р':
    lst_drop.append(i)
lst_test_drop = []
for i in df_test.columns:
  if i[0] + i[1] == 'id' or i[0] == 'Р':
    lst_test_drop.append(i)

In [ ]:
lst_drop.append("type");lst_drop.append('job');lst_drop.append('division')

In [ ]:
lst_test_drop.append('type');#lst_test_drop.append('job');lst_test_drop.append('division')

In [ ]:
lst_test_drop

['id', 'Руководитель_x', 'Руководитель', 'type']

In [ ]:
cat = []
for i in df_train.columns:
  print(i, i[0] == 'e' or i == 'mode_task_type_y' or i == 'mode_task_type_x')
  if i[0] == 'e' or i == 'mode_task_type_y' or i == 'mode_task_type_x':
    cat.append(i)

id False
type False
division False
job False
count_ed_x False
ed_type_1_x True
ed_type_2_x True
ed_type_3_x True
ed_spec_0_x True
ed_spec_1_x True
ed_spec_2_x True
ed_spec_3_x True
ed_spec_4_x True
ed_spec_5_x True
Длительность_outliers_x False
Длительность_std_x False
Длительность_med_x False
is_outlier_Calls_x False
c_T_x False
s_C_x False
c_C_x False
CallTime_sum_x False
CallTime_median_x False
CallTime_count_x False
CallTime_std_x False
Количество задач_x False
Просрочено дней_x False
Статус по просрочке_x False
mode_task_type_x True
type04tasks_Акт_x False
type04tasks_Внутренний документ_x False
type04tasks_Внутренний документ для БП синхронизация 1С_x False
type04tasks_Входящий_x False
type04tasks_Договор_x False
type04tasks_Дополнительное соглашение_x False
type04tasks_Запрос на внесение изменений_x False
type04tasks_Исходящий_x False
type04tasks_Исходящий РМЗ_x False
type04tasks_Отчет по ВА_x False
type04tasks_Приказ_x False
type04tasks_Протокол_x False
type04tasks_Распоряжение

In [ ]:
cat

['ed_type_1_x',
 'ed_type_2_x',
 'ed_type_3_x',
 'ed_spec_0_x',
 'ed_spec_1_x',
 'ed_spec_2_x',
 'ed_spec_3_x',
 'ed_spec_4_x',
 'ed_spec_5_x',
 'mode_task_type_x',
 'mode_task_type_y',
 'ed_type_1_y',
 'ed_type_2_y',
 'ed_type_3_y',
 'ed_spec_0_y',
 'ed_spec_1_y',
 'ed_spec_2_y',
 'ed_spec_3_y',
 'ed_spec_4_y',
 'ed_spec_5_y']

In [ ]:
cnt = 0
for i in df_train.columns:
  for j in cat:
    if i==j:
      cnt+=1
cnt, len(cat)

(20, 20)

In [ ]:
df_train[cat] = df_train[cat].astype('string') 
df_test[cat] = df_test[cat].astype('string') 

In [ ]:
from sklearn import preprocessing

In [ ]:
for i in tmp.columns:
  print(i)

Руководитель
ed_type_1
ed_spec_1


In [ ]:
# tmp = df_train.drop(cat, axis=1).drop(lst_drop, axis=1)
# scaler = preprocessing.StandardScaler().fit(tmp)
# df_train[tmp.columns] = scaler.transform(tmp)

In [ ]:
lst_drop

['id', 'Руководитель_x', 'Руководитель', 'type', 'job', 'division']

In [ ]:
tmp=df_train.drop(lst_drop[:-3], axis=1).drop_duplicates()

In [ ]:
tmp.columns

Index(['type', 'division', 'job', 'count_ed_x', 'ed_type_1_x', 'ed_type_2_x',
       'ed_type_3_x', 'ed_spec_0_x', 'ed_spec_1_x', 'ed_spec_2_x',
       ...
       'count_ed_y', 'ed_type_1_y', 'ed_type_2_y', 'ed_type_3_y',
       'ed_spec_0_y', 'ed_spec_1_y', 'ed_spec_2_y', 'ed_spec_3_y',
       'ed_spec_4_y', 'ed_spec_5_y'],
      dtype='object', length=179)

In [ ]:
tmp = tmp.drop(tmp.loc[tmp['type'] == 2].index[:round(len(tmp.loc[tmp['type'] == 2].index)/2.5)])

In [ ]:
X = tmp.drop(['type', 'job', 'division'], axis=1)
X_t = df_test.drop([i.strip() for i in lst_test_drop], axis = 1)
y = tmp[['job', 'division',]] # 

In [ ]:
print(*(i for i in X.columns))

count_ed_x ed_type_1_x ed_type_2_x ed_type_3_x ed_spec_0_x ed_spec_1_x ed_spec_2_x ed_spec_3_x ed_spec_4_x ed_spec_5_x Длительность_outliers_x Длительность_std_x Длительность_med_x is_outlier_Calls_x c_T_x s_C_x c_C_x CallTime_sum_x CallTime_median_x CallTime_count_x CallTime_std_x Количество задач_x Просрочено дней_x Статус по просрочке_x mode_task_type_x type04tasks_Акт_x type04tasks_Внутренний документ_x type04tasks_Внутренний документ для БП синхронизация 1С_x type04tasks_Входящий_x type04tasks_Договор_x type04tasks_Дополнительное соглашение_x type04tasks_Запрос на внесение изменений_x type04tasks_Исходящий_x type04tasks_Исходящий РМЗ_x type04tasks_Отчет по ВА_x type04tasks_Приказ_x type04tasks_Протокол_x type04tasks_Распоряжение_x type04tasks_Служебная записка_x type04tasks_Файл_x count_x Количество делегированных задач_x activeTime_outliers_x monitorTime_outliers_x activeTime_sum_x activeTime_median_x activeTime_count_x activeTime_std_x monitorTime_sum_x monitorTime_median_x moni

In [ ]:
df_test.drop('Руководитель_x', axis=1)

,id,type,count_ed_x,ed_type_1_x,ed_type_2_x,ed_type_3_x,ed_spec_0_x,ed_spec_1_x,ed_spec_2_x,ed_spec_3_x,...,ed_type_1_y,ed_type_2_y,ed_type_3_y,Руководитель,ed_spec_0_y,ed_spec_1_y,ed_spec_2_y,ed_spec_3_y,ed_spec_4_y,ed_spec_5_y
0,ОРГ1-02649,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ОРГ2-05929,0,1.0,Высшее образование,0,0,Инженерно-геодезическое обеспечение и техничес...,0,0,0,...,Среднее профессиональное образование,0,0,ОРГ2-00539,"Строительство железных дорог, путь и путевое х...",0,0,0,0,0
2,ОРГ2-05859,0,1.0,Среднее общее образование,0,0,0,0,0,0,...,Высшее образование,0,0,ОРГ2-04561,0,0,0,0,0,0
3,ОРГ1-00279,0,2.0,Повышение квалификации,Высшее образование,0,0,0,0,0,...,Аспирантура,Повышение квалификации,Высшее образование,ОРГ1-00483,Интегрированная система менеджмента согласно с...,0,0,0,0,0
4,ОРГ2-01613,0,1.0,Высшее образование,0,0,Промышленное и гражданское строительство,0,0,0,...,Высшее образование,0,0,ОРГ2-02458,Водоснабжение и водоотведение,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,ОРГ1-02893,0,0.0,Среднее профессиональное образование,0,0,Менеджмент,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258,ОРГ2-01077,0,1.0,Высшее образование,0,0,Астрономогеодезия,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259,ОРГ1-00469,0,1.0,Высшее образование,0,0,Менеджмент организации,0,0,0,...,Высшее образование,0,0,ОРГ1-01974,Прикладная математика и информатика,0,0,0,0,0
260,ОРГ1-02060,0,0.0,Высшее образование,0,0,"Проектирование, сооружение и эксплуатация газо...",0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_t.shape

(262, 176)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381 entries, 2 to 548
Columns: 176 entries, count_ed_x to ed_spec_5_y
dtypes: float64(156), string(20)
memory usage: 526.9 KB


In [ ]:
# X[['Вид образования_x', 'Специальность_x', 'Вид образования_y', 'Специальность_y']] = X[['Вид образования_x', 'Специальность_x', 'Вид образования_y', 'Специальность_y']].astype('string')

In [ ]:

from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Обучение модели

In [ ]:
tmp['job'].value_counts()

1    294
0     87
Name: job, dtype: int64

In [ ]:
tmp['division'].value_counts()

1    222
0    159
Name: division, dtype: int64

In [ ]:
params = {                  # 'task_type':'GPU',
                            'iterations':800,
                            'depth':7,
                            'random_state':SEED,
                            'learning_rate':0.02,
                            'eval_metric':'HammingLoss',
                            'loss_function':'MultiLogloss'}

In [ ]:
clf_local = cb.CatBoostClassifier(**params)

In [ ]:
pool_train = cb.Pool(X_train, y_train, cat_features=cat)

In [ ]:
pool_test = cb.Pool(X_test, y_test, cat_features=cat)

In [ ]:
submit_pool_train = cb.Pool(X, y, cat_features=cat)

In [ ]:
submit_pool_test = cb.Pool(X_t, cat_features=cat)

In [ ]:
# clf_local.fit(pool_train)

## Оценка точности

In [ ]:
def make_full_pred(pred_job, pred_div):
  full_pred = []
  assert len(pred_job)==len(pred_div)
  for i in range(len(pred_div)):
    tmp = {'div':pred_div[i], 'job':pred_job[i]}
    if tmp['job'] == 1 and tmp['div'] == 0:
      full_pred.append(0)
    elif tmp['job'] == 0 and tmp['div'] == 1:
      full_pred.append(1)
    elif tmp['job'] == 1 and tmp['div'] == 1:
      full_pred.append(2)
    else: full_pred.append(3)
  return full_pred

In [ ]:
# pred_proba = clf_local.predict_proba(pool_test)
# pred_job = [0 if i[0] < 0.7 else 1 for i in pred_proba]
# pred_div = [0 if i[1] < 0.04 else 1 for i in pred_proba]
# pred = make_full_pred(pred_job, pred_div)
# full_y_test = make_full_pred(list(y_test['job']), list(y_test['division']))

In [ ]:
# y_test['preds'] = pred
# y_test['preds'].value_counts()

In [ ]:
# for i in range(len(pred)):
  # if pred[i] == full_y_test[i] == 3:
    # print('thats it')

In [ ]:
# pred

In [ ]:
# from sklearn.metrics import recall_score

# print("Recall score:", recall_score(y_test['job'], pred_job, average='macro'))
# print("Recall score:", recall_score(y_test['division'], pred_div, average='macro'))
# print("Recall score:", recall_score(full_y_test, pred, average='macro'))

In [ ]:
clf_sub = cb.CatBoostClassifier(**params)
clf_sub.fit(submit_pool_train)

0:	learn: 0.2742782	total: 70.6ms	remaining: 56.4s
1:	learn: 0.2559055	total: 231ms	remaining: 1m 32s
2:	learn: 0.2467192	total: 343ms	remaining: 1m 31s
3:	learn: 0.2217848	total: 485ms	remaining: 1m 36s
4:	learn: 0.2217848	total: 596ms	remaining: 1m 34s
5:	learn: 0.2112861	total: 719ms	remaining: 1m 35s
6:	learn: 0.2112861	total: 839ms	remaining: 1m 34s
7:	learn: 0.2152231	total: 989ms	remaining: 1m 37s
8:	learn: 0.2086614	total: 1.1s	remaining: 1m 36s
9:	learn: 0.2073491	total: 1.24s	remaining: 1m 37s
10:	learn: 0.2073491	total: 1.35s	remaining: 1m 36s
11:	learn: 0.2086614	total: 1.49s	remaining: 1m 38s
12:	learn: 0.2152231	total: 1.62s	remaining: 1m 38s
13:	learn: 0.2125984	total: 1.64s	remaining: 1m 32s
14:	learn: 0.2086614	total: 1.78s	remaining: 1m 33s
15:	learn: 0.2125984	total: 1.92s	remaining: 1m 33s
16:	learn: 0.2152231	total: 1.97s	remaining: 1m 30s
17:	learn: 0.2099738	total: 2.1s	remaining: 1m 31s
18:	learn: 0.2073491	total: 2.2s	remaining: 1m 30s
19:	learn: 0.2020997	tota

In [ ]:
pred_proba = clf_sub.predict_proba(submit_pool_test)

In [ ]:
real_test = pd.read_csv('/content/test.csv')

In [ ]:
def get_tresholds(pred_proba):
  min_error = 1
  min_t1 = 1
  min_t2 = 1
  for i in range(100, 200):
    for j in range(100, 200):
      t1 = i/200
      t2 = j/200
      pred_job = [0 if i[0] < t1 else 1 for i in pred_proba]
      pred_div = [0 if i[1] < t2 else 1 for i in pred_proba]
      pred = make_full_pred(pred_job, pred_div)
      real_test['type'] = pred
      local_tmp = (real_test['type'].value_counts() / real_test['type'].count())
      m = local_tmp.median()
      error = abs(m - local_tmp.mean()) + m +  local_tmp.std() + 2*(local_tmp.max() - local_tmp.min())
      if error < min_error:
        min_t1 = t1
        min_t2 = t2
        min_error = error
  return min_t1, min_t2, min_error

In [ ]:
min_t1, min_t2, min_error = get_tresholds(pred_proba)

In [ ]:
min_t1, min_t2, min_error

(0.905, 0.65, 0.47507501952138503)

In [ ]:
pred_job = [0 if i[0] < min_t1 else 1 for i in pred_proba]
pred_div = [0 if i[1] < min_t2 else 1 for i in pred_proba]
pred = make_full_pred(pred_job, pred_div)

In [ ]:

real_test['type'] = pred

In [ ]:
(real_test['type'].value_counts() / real_test['type'].count()).mean()

0.25

In [ ]:
(real_test['type'].value_counts() / real_test['type'].count())

0    0.297710
1    0.270992
3    0.225191
2    0.206107
Name: type, dtype: float64

In [ ]:
list(df_full.columns)

['count_ed_x',
 'ed_type_1_x',
 'ed_type_2_x',
 'ed_type_3_x',
 'ed_spec_0_x',
 'ed_spec_1_x',
 'ed_spec_2_x',
 'ed_spec_3_x',
 'ed_spec_4_x',
 'ed_spec_5_x',
 'Длительность_outliers_x',
 'Длительность_std_x',
 'Длительность_med_x',
 'is_outlier_Calls_x',
 'c_T_x',
 's_C_x',
 'c_C_x',
 'CallTime_sum_x',
 'CallTime_median_x',
 'CallTime_count_x',
 'CallTime_std_x',
 'Количество задач_x',
 'Просрочено дней_x',
 'Статус по просрочке_x',
 'mode_task_type_x',
 'type04tasks_Акт_x',
 'type04tasks_Внутренний документ_x',
 'type04tasks_Внутренний документ для БП синхронизация 1С_x',
 'type04tasks_Входящий_x',
 'type04tasks_Договор_x',
 'type04tasks_Дополнительное соглашение_x',
 'type04tasks_Запрос на внесение изменений_x',
 'type04tasks_Исходящий_x',
 'type04tasks_Исходящий РМЗ_x',
 'type04tasks_Отчет по ВА_x',
 'type04tasks_Приказ_x',
 'type04tasks_Протокол_x',
 'type04tasks_Распоряжение_x',
 'type04tasks_Служебная записка_x',
 'type04tasks_Файл_x',
 'count_x',
 'Количество делегированных зад

In [ ]:
df_full['id'] = df_full.index

In [ ]:
df_full['type'] = 3
df_full['job'] = 0
df_full['division'] = 0

In [ ]:
# for i i

In [ ]:
aux_tmp = df_full.drop(lst_drop, axis=1)
aux_tmp = aux_tmp.fillna(0)
aux_tmp[cat] = aux_tmp[cat].astype('string')
X_auxiliary = cb.Pool(aux_tmp, cat_features=cat)
id_auxiliary = df_full.index
pred_proba_auxiliary = clf_sub.predict_proba(X_auxiliary)
# N = 400 # топ 400 минимальных по вероятности 3его класса 
tmp = pd.DataFrame()
f = lambda x, k: [i[k] for i in x]
tmp['proba_1'] = f(pred_proba_auxiliary, 0) 
tmp['proba_2'] = f(pred_proba_auxiliary, 1) 
tmp['proba_1'] = tmp['proba_1'] < min_t1-0.1
tmp['proba_2'] = tmp['proba_2'] < min_t2-0.1
idxs = tmp.loc[tmp['proba_1'] & tmp['proba_2']].index
auxiliary = df_full.iloc[idxs]

In [ ]:
auxiliary['type'] = 3
auxiliary['job'] = 0
auxiliary['division'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
X_train = pd.concat([df_train, auxiliary], axis=0)

In [ ]:
X_train

,id,type,division,job,count_ed_x,ed_type_1_x,ed_type_2_x,ed_type_3_x,ed_spec_0_x,ed_spec_1_x,...,ed_type_1_y,ed_type_2_y,ed_type_3_y,Руководитель,ed_spec_0_y,ed_spec_1_y,ed_spec_2_y,ed_spec_3_y,ed_spec_4_y,ed_spec_5_y
0,ОРГ1-02050,2,1,1,0.0,Высшее образование,Среднее профессиональное образование,0,Технологические машины и оборудование,0,...,0,0,0,0,0,0,0,0,0,0
1,ОРГ1-02783,2,1,1,1.0,Высшее образование,0,0,Автоматизация технологических процессов и прои...,0,...,Высшее образование,0,0,ОРГ1-01974,Прикладная математика и информатика,0,0,0,0,0
2,ОРГ2-06173,0,0,1,1.0,Высшее образование,0,0,Управление качеством,0,...,Высшее образование,0,0,ОРГ2-01170,Правовое обеспечение государственного и муници...,0,0,0,0,0
3,ОРГ2-05359,2,1,1,1.0,Среднее профессиональное образование,0,0,Технолог-сварщик,0,...,0,0,0,0,0,0,0,0,0,0
4,ОРГ1-00803,2,1,1,0.0,Высшее образование,0,0,Журналистика,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ОРГ2-02371,ОРГ2-02371,3,0,0,0.0,Высшее образование,0,0,Менеджмент организации,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ОРГ2-04904,ОРГ2-04904,3,0,0,0.0,Высшее образование,0,0,Государственное и муниципальное управление,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ОРГ2-05032,ОРГ2-05032,3,0,0,1.0,Высшее образование,0,0,Социально-культурный сервис и туризм,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ОРГ2-05141,ОРГ2-05141,3,0,0,1.0,Высшее образование,0,0,Моделирование и исследование операций в органи...,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X_train['division'] = X_train['type'].apply(change_division)
X_train['job'] = X_train['type'].apply(change_job)
y_train = X_train[['job', 'division']].fillna(0)
aux_tmp = X_train.drop(lst_drop, axis=1)
aux_tmp = aux_tmp.fillna(0)
aux_tmp[cat] = aux_tmp[cat].astype('string')
pool_train = cb.Pool(aux_tmp, y_train, cat_features=cat)

In [ ]:
params = {                  # 'task_type':'GPU',
                            'iterations':1200,
                            'depth':7,
                            'random_state':SEED,
                            'learning_rate':0.01,
                            'eval_metric':'HammingLoss',
                            'loss_function':'MultiCrossEntropy'}
clf_with_aux = cb.CatBoostClassifier(**params)

In [ ]:
lst_drop

['id', 'Руководитель_x', 'Руководитель', 'type', 'job', 'division']

In [ ]:

# aux_tmp = aux_tmp.drop(['division', 'job'])

In [ ]:
aux_tmp.columns

Index(['count_ed_x', 'ed_type_1_x', 'ed_type_2_x', 'ed_type_3_x',
       'ed_spec_0_x', 'ed_spec_1_x', 'ed_spec_2_x', 'ed_spec_3_x',
       'ed_spec_4_x', 'ed_spec_5_x',
       ...
       'count_ed_y', 'ed_type_1_y', 'ed_type_2_y', 'ed_type_3_y',
       'ed_spec_0_y', 'ed_spec_1_y', 'ed_spec_2_y', 'ed_spec_3_y',
       'ed_spec_4_y', 'ed_spec_5_y'],
      dtype='object', length=176)

In [ ]:
idxs

Int64Index([   4,   21,   43,   44,   45,   65,  144,  164,  198,  249,  263,
             298,  395,  423,  438,  463,  500,  503,  505,  507,  508,  509,
             529,  577,  582,  611,  620,  621,  632,  657,  668,  723,  731,
             768,  813,  814,  871,  875,  883,  901,  937,  966,  982,  988,
            1006, 1086, 1089, 1095, 1097, 1143, 1144, 1211, 1216, 1227, 1326,
            1423, 1612, 1789, 2372, 2392, 2416, 3760],
           dtype='int64')

In [ ]:
clf_with_aux.fit(pool_train)

0:	learn: 0.2295752	total: 148ms	remaining: 2m 57s
1:	learn: 0.2124183	total: 278ms	remaining: 2m 46s
2:	learn: 0.2034314	total: 411ms	remaining: 2m 43s
3:	learn: 0.2009804	total: 529ms	remaining: 2m 38s
4:	learn: 0.1911765	total: 678ms	remaining: 2m 42s
5:	learn: 0.1936275	total: 806ms	remaining: 2m 40s
6:	learn: 0.1985294	total: 832ms	remaining: 2m 21s
7:	learn: 0.1903595	total: 959ms	remaining: 2m 22s
8:	learn: 0.1936275	total: 1s	remaining: 2m 12s
9:	learn: 0.1870915	total: 1.14s	remaining: 2m 15s
10:	learn: 0.1911765	total: 1.27s	remaining: 2m 17s
11:	learn: 0.1903595	total: 1.41s	remaining: 2m 19s
12:	learn: 0.1879085	total: 1.52s	remaining: 2m 19s
13:	learn: 0.1944444	total: 1.57s	remaining: 2m 12s
14:	learn: 0.1936275	total: 1.71s	remaining: 2m 15s
15:	learn: 0.1936275	total: 1.85s	remaining: 2m 17s
16:	learn: 0.1944444	total: 2.01s	remaining: 2m 19s
17:	learn: 0.1968954	total: 2.14s	remaining: 2m 20s
18:	learn: 0.1968954	total: 2.32s	remaining: 2m 24s
19:	learn: 0.1968954	tota

In [ ]:
pred_proba = clf_with_aux.predict_proba(submit_pool_test)

In [ ]:
min_t1, min_t2, min_error = get_tresholds(pred_proba)

In [ ]:
min_t1, min_t2, min_error

(0.875, 0.71, 0.3072519083969465)

In [ ]:
# min_t1, min_t2, min_error

In [ ]:
pred_job = [0 if i[0] < min_t1 else 1 for i in pred_proba]
pred_div = [0 if i[1] < min_t2 else 1 for i in pred_proba]
pred = make_full_pred(pred_job, pred_div)

In [ ]:
real_test['type'] = pred

In [ ]:
(real_test['type'].value_counts() / real_test['type'].count())

3    0.263359
2    0.255725
0    0.240458
1    0.240458
Name: type, dtype: float64

In [ ]:
real_test.to_csv('submit_якутск.csv', index=False)

In [ ]:
!cp /content/submit_якутск.csv /content/drive/MyDrive/submit_Якутск.csv

In [ ]:
print(*(i for i in X_train.columns))

id type division job count_ed_x ed_type_1_x ed_type_2_x ed_type_3_x ed_spec_0_x ed_spec_1_x ed_spec_2_x ed_spec_3_x ed_spec_4_x ed_spec_5_x Длительность_outliers_x Длительность_std_x Длительность_med_x is_outlier_Calls_x c_T_x s_C_x c_C_x CallTime_sum_x CallTime_median_x CallTime_count_x CallTime_std_x Количество задач_x Просрочено дней_x Статус по просрочке_x mode_task_type_x type04tasks_Акт_x type04tasks_Внутренний документ_x type04tasks_Внутренний документ для БП синхронизация 1С_x type04tasks_Входящий_x type04tasks_Договор_x type04tasks_Дополнительное соглашение_x type04tasks_Запрос на внесение изменений_x type04tasks_Исходящий_x type04tasks_Исходящий РМЗ_x type04tasks_Отчет по ВА_x type04tasks_Приказ_x type04tasks_Протокол_x type04tasks_Распоряжение_x type04tasks_Служебная записка_x type04tasks_Файл_x count_x Количество делегированных задач_x activeTime_outliers_x monitorTime_outliers_x activeTime_sum_x activeTime_median_x activeTime_count_x activeTime_std_x monitorTime_sum_x moni

In [ ]:
sorted(list(zip(X_train.drop(lst_drop, axis=1).columns, np.array(clf_sub.feature_importances_).ravel())), key=lambda tpl: tpl[1], reverse=True)

[('mode_task_type_x', 4.05167068306551),
 ('ed_spec_0_y', 3.679083280170882),
 ('type04tasks_Распоряжение_x', 3.603388730442207),
 ('ed_type_2_x', 2.688698694341232),
 ('Время опоздания_count_x', 2.578701677478341),
 ('Время опоздания_std_x', 2.5473446402026116),
 ('Время опоздания_median_x', 2.418607770176146),
 ('monitor_Time_std_x', 2.2225387849808755),
 ('Время опоздания_sum_x', 2.2054885222201595),
 ('ed_type_1_x', 2.122066438847289),
 ('type04tasks_Входящий_x', 2.1033387800516823),
 ('mode_task_type_y', 1.9662360269478114),
 ('monitorTime_median_x', 1.8749558044056747),
 ('activeTime_sum_x', 1.8532358378980898),
 ('monitor_Time_median_x', 1.7811310398719808),
 ('activeTime_median_x', 1.7623654948960552),
 ('frequency_monitor_Time_x', 1.7398538277012916),
 ('activeTime_std_x', 1.6319995464680999),
 ('ed_type_1_y', 1.4259669864228937),
 ('monitor_Time_count_x', 1.3911794463918294),
 ('Mean_Время_опоздания_by_ed_Руководитель_median_x', 1.3669623860113278),
 ('monitorTime_sum_x', 1.3